# SNLP Assignment 5 

Name 1: William LaCroix<br/>
Student id 1: 7038732<br/>
Email 1: williamplacroix@gmail.com<br/>


Name 2: Nicholas Jennings<br/>
Student id 2: 2573492<br/>
Email 2: s8nijenn@stud.uni-saarland.de<br/>

**Instructions:** Read each question carefully. <br/>
Make sure you appropriately comment your code wherever required. Your final submission should contain the completed Notebook and the respective Python files for any additional exercises necessary. There is no need to submit the data files should they exist. <br/>
Upload the zipped folder on CMS. Please follow the naming convention of **Name1_studentID1_Name2_studentID2.zip**. Make sure to click on "Turn-in" (or the equivalent on CMS) after your upload your submission, otherwise the assignment will not be considered as submitted. Only one member of the group should make the submisssion.

---

# Exercise 1: Back-Off and Interpolation (5 points) </br>

In Assignment 4, we implemented a simple smoothing technique by presuming counts where they did not exist. A more informed version of smoothing is to look into lower order n-grams when we see novel word combinations. There are two popular ways of doing this: Katz backoff and linear interpolation. 


1. Look these terms up and describe how they differ in 3-4 sentences (0.5 points)

2. We generally use cross validation to determine the hyperparameters used in these models. Describe k-fold cross validation in your own words (0.5 points)

3. Let's implement an interpolated ngram model. Write a Language model class that trains an ngram model on a training corpus ```(List[List[str]])``` and computes the perplexity on a test corpus ```(List[List[str]])```. Use add-$\alpha$ smoothing and interpolation weights of $\lambda_i=\frac{1}{n}, \forall i \in \{1\dots n\}$ where $n$ is the order of the model. As usual, you will need to account for OOV tokens with ```<unk>```. Use the top_5000 words for vocabulary. You may reuse code from Asignment 4. (2 points)

4. Find the optimal $\alpha$ for smoothing and the optimal n-gram order using 5-fold cross validation (independently).  (2 points)

We expect you to make some design choices yourself. Please document them appropriately.

In [23]:
import exercise_1
from importlib import reload
exercise_1 = reload(exercise_1)

corpus = exercise_1.load_and_preprocess_data()


#TODO: use a 70:30 split for train/test. Take test from the last 30%. Do not randomize at this stage.

#train,test = split_corpus(corpus)


#TODO: Finish this model. Pass necessary arguments
#model = exercise_1.InterpolatedModel(args)

#Calculate test perplexity
#Interpolatedmodel.perplexity()

#implement cross validation to find optimal n and alpha. You might want to do this independently.  
#Plot perplexity vs alpha graph.
#Plot perplexity vs n graph. 
#Write up conclusions. 


In [24]:
for sent in corpus:
    print(sent)

['pierre', 'vinken', '61', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov', '29']
['mr', 'vinken', 'is', 'chairman', 'of', 'elsevier', 'nv', 'the', 'dutch', 'publishing', 'group']
['rudolph', 'agnew', '55', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', '1', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate']
['a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of', 'workers', 'exposed', 'to', 'it', 'more', 'than', '30', 'years', 'ago', 'researchers', 'reported', '0', 't1']
['the', 'asbestos', 'fiber', 'crocidolite', 'is', 'unusually', 'resilient', 'once', 'it', 'enters', 'the', 'lungs', 'with', 'even', 'brief', 'exposures', 'to', 'it', 'causing', 'symptoms', 'that', 't1', 'show', 'up', 'decades', 'later', 'resear

## Exercise 2: Kneser-Ney Smoothing (5 points)


Consider the following notation


- $V$ - LM vocabulary
-  $N (x)$ - count of the n-gram x in the training corpus 
- $N_+(\bullet w) \triangleq |\{u : N (u, w) > 0\}|$ - number of bigram types ending in w
- $N_+(w \bullet) \triangleq |\{u : N (w, u) > 0\}|$ - number of bigram types starting with w
- $N_+(\bullet w \bullet) \triangleq |\{(u,v) : N (u, w, u) > 0\}|$ - number of trigram types with w in the middle

This exercise aims to provide a basic understanding of Kneser-Ney Smoothing. Kneser-Ney Smoothing makes use of *continuation counts* of words for lower order n-grams, given as

\begin{equation}
C_{KN} = 
\begin{cases}
\text{count}(\bullet) & \text{for highest order} \\
\text{continuationcount}(\bullet) & \text{for lower orders}
\end{cases}
\end{equation}


For a trigram model, Kneser-Ney Smoothing is implemented using the following equations:

$$P_{KN}(w_3|w_1, w_2) = \frac{\max\{N(w_1 w_2 w_3)-d,0\}}{N(w_1 w_2)} + \lambda(w_1, w_2)P_{KN}(w_3|w_2)$$

$$P_{KN}(w_3|w_2) = \frac{\max\{N_{+}(\bullet w_2 w_3)-d,0\}}{N_{+}(\bullet w_2 \bullet)} + \lambda(w_2)P_{KN}(w_3)$$

\begin{equation}
P_{KN}(w_3) = \begin{cases}
\frac{N_{+}(\bullet w_3)}{N_{+}(\bullet \bullet)} & \text{if $w_3 \in$ V} \\
\frac{1}{V} & \text{otherwise}
\end{cases}
\end{equation}

$\lambda$ is used to normalise the discounted probability mass and is given by

$$\lambda(w_1, w_2) = \frac{d}{N(w_1 w_2)} \cdot N_{+}(w_1 w_2 \bullet)$$

$$\lambda(w_2) = \frac{d}{N(w_2)} \cdot N_{+}(w_2 \bullet)$$


1. Explain the main intuition behind Kneser-Ney language models and discuss how it is
different from absolute discounting LMs [1 point]

2. Assume that we make a trigram model from [alice_in_wonderland.txt](https://gist.github.com/phillipj/4944029). Preprocess the text by first removing all punctuation, lowercasing, and then tokenisation. There is no need to split the data into train and test sets. (0.5 points)

3. Code a function that calculates the bigram probability $P_{KN}(w_3|w_2)$ using the equations provided. The discounting parameter d = 0.75. Use your function to calculate the probabilities $P_{KN}(nothing|said)$ and $P_{KN}(nichts|said)$. (2 points)

4. Manually compute the MLE and the Add-1 probabilities for the same bigrams. How do these compare to Kneser-Ney smoothing? (1 point)

In [10]:
from importlib import reload
import exercise_2
exercise_2=reload(exercise_2)

text=open('alice_in_wonderland.txt').read()
tokenized_text=exercise_2.preprocess_text(text)
print(tokenized_text[:100])


#Feel free to not use a class. This is just boilerplate.
#BigramCounter = exercise_2.KneserNeyCounter(args)
#BigramCounter.prob(bigram) 


['alices', 'adventures', 'in', 'wonderland', 'alices', 'adventures', 'in', 'wonderland', 'lewis', 'carroll', 'the', 'millennium', 'fulcrum', 'edition', '30', 'chapter', 'i', 'down', 'the', 'rabbithole', 'alice', 'was', 'beginning', 'to', 'get', 'very', 'tired', 'of', 'sitting', 'by', 'her', 'sister', 'on', 'the', 'bank', 'and', 'of', 'having', 'nothing', 'to', 'do', 'once', 'or', 'twice', 'she', 'had', 'peeped', 'into', 'the', 'book', 'her', 'sister', 'was', 'reading', 'but', 'it', 'had', 'no', 'pictures', 'or', 'conversations', 'in', 'it', 'and', 'what', 'is', 'the', 'use', 'of', 'a', 'book', 'thought', 'alice', 'without', 'pictures', 'or', 'conversation', 'so', 'she', 'was', 'considering', 'in', 'her', 'own', 'mind', 'as', 'well', 'as', 'she', 'could', 'for', 'the', 'hot', 'day', 'made', 'her', 'feel', 'very', 'sleepy', 'and']


# Bonus Question: Good Turing Smoothing (2 points)


We know that Add-alpha discounting is not very efficient since it takes away too much probability mass for distribution. A better alternative is Good-Turing smoothing, whoch distributes probbaility mass for a lower order ngram from immediately higher ngrams. 

1. Describe the central idea behind Good-Turing smoothing and state the formulas for Good Turing counts and Good Turing probability. (0.5)

2. Given some fixed history, How much probability mass is set aside when we use Good Turing counts? Express this mathematically. (1 point) 

   
3. What is the main problem with Good Turing estimation? (0.5)
